# PyCitySchools Standardized Testing Report

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [22]:
School_data_by_school = school_data_complete.set_index("school_name")


In [3]:
# Creating all the variable required for the first plot

total_schools = school_data_complete["school_name"].nunique()
total_students = school_data_complete["Student ID"].count()
total_budget = school_data["budget"].sum()
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()

# Get percent of Students who passed math
math_pass_df = school_data_complete.loc[school_data_complete["math_score"]>=70]
math_pass_number = math_pass_df["student_name"].count()
math_percent_pass = (math_pass_number/total_students)*100

# Get Percent of Students who passed reading
reading_pass_df = school_data_complete.loc[school_data_complete["reading_score"]>=70]
reading_pass_number = reading_pass_df["student_name"].count()
reading_percent_pass = (reading_pass_number/total_students)*100

# Get the Percent of Students who pass both reading and math
math_reading_pass_df = school_data_complete.loc[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70),:]
math_reading_pass_number = math_reading_pass_df["student_name"].count()
math_reading_percent_pass = (math_reading_pass_number/total_students)*100


In [4]:
# make sure I add the formating to the code bellow

## District Summary

In [5]:
# Creating the dataframe and dictionary for the school disctrict summary

district_summary_df = pd.DataFrame(
    {"Total Schools" : [total_schools], "Total Students": [total_students], 
     "Total Budget": [total_budget], "Average Math Score": [average_math_score], 
    "Average Math Score": [average_math_score], 
     "Average Reading Score": [average_reading_score], "% Passing Math": [math_percent_pass], 
    "% Passing Reading": [reading_percent_pass], "% Overall Passing" : [math_reading_percent_pass]})

#Formating
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)


district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [6]:
student_per_school = school_data_complete["school_name"].value_counts()
student_per_school_df = pd.DataFrame(student_per_school)
student_per_school_df = student_per_school_df.reset_index()
student_per_school_df.columns = ['school_name', 'Total Students']
# student_per_school_df.dtypes

In [7]:
# find sum the math scores for each school
# divied by the number of student at that school then multiply by 100
means_by_school= school_data_complete.groupby("school_name").mean()
means_by_school.drop(['School ID','Student ID', 'budget', 'size'], inplace=True, axis=1)

In [23]:
count_of_passing_math = math_pass_df.groupby("school_name")["type"].count()
count_of_passing_reading = reading_pass_df.groupby("school_name")["type"].count()
count_of_passing_reading_math = math_reading_pass_df.groupby("school_name")["type"].count()
merged_counts = pd.DataFrame({
    "Number of students Passing Math" : count_of_passing_math,
    "Number of students Passing Reading": count_of_passing_reading,
    "Number of students Passing Math and Reading": count_of_passing_reading_math
})


In [9]:
school_summary = pd.merge(school_data,student_per_school_df, how = 'left', on=["school_name"])
budget_per_student = school_summary["budget"]/school_summary["Total Students"]
school_summary["budget per student"] = budget_per_student
school_summary_df= pd.merge(school_summary, merged_counts, how = "left", on = "school_name")

In [10]:
percent_passing_math = (school_summary_df["Number of students Passing Math"]/school_summary_df["Total Students"])*100
school_summary_df["% passing math"] = percent_passing_math

percent_passing_reading = (school_summary_df["Number of students Passing Reading"]/school_summary_df["Total Students"])*100
school_summary_df["% passing reading"] = percent_passing_reading

percent_passing_math_reading = (school_summary_df["Number of students Passing Math and Reading"]/school_summary_df["Total Students"])*100
school_summary_df["% overall passing"] = percent_passing_math_reading

school_summary_df.drop(["School ID", "size", "Number of students Passing Math", "Number of students Passing Reading", "Number of students Passing Math and Reading"], inplace = True, axis=1 )
school_summary_df_2 = school_summary_df.merge (means_by_school, on = "school_name")

In [11]:
# Formating
school_summary_df_2.columns = ["school_name", "School Type", "Total School Budget","Student Population", "Per Student Budget", "% Passing Math", "% Passing Reading","% Overall Passing", "Average Reading Score", "Average Math Score"]

## School Summary

In [12]:
school_summary = school_summary_df_2.set_index("school_name")
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)


school_summary

,School Type,Total School Budget,Student Population,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Reading Score,Average Math Score
school_name,,,,,,,,,
Huang High School,District,"$1,910,635.00",2917,$655.00,65.683922,81.316421,53.513884,81.182722,76.629414
Figueroa High School,District,"$1,884,411.00",2949,$639.00,65.988471,80.739234,53.204476,81.158020,76.711767
Shelton High School,Charter,"$1,056,600.00",1761,$600.00,93.867121,95.854628,89.892107,83.725724,83.359455
Hernandez High School,District,"$3,022,020.00",4635,$652.00,66.752967,80.862999,53.527508,80.934412,77.289752
Griffin High School,Charter,"$917,500.00",1468,$625.00,93.392371,97.138965,90.599455,83.816757,83.351499
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,93.867718,96.539641,90.582567,83.989488,83.274201
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,94.133477,97.039828,91.334769,83.975780,83.061895
Bailey High School,District,"$3,124,928.00",4976,$628.00,66.680064,81.933280,54.642283,81.033963,77.048432
Holden High School,Charter,"$248,087.00",427,$581.00,92.505855,96.252927,89.227166,83.814988,83.803279


## Top 5 Performing Schools by Overall Students Passing

In [13]:
# Sort and post head and tail
sortedvalues = school_summary_df_2.sort_values("% Overall Passing", ascending = False).set_index("school_name")
sortedvalues["Per Student Budget"] = sortedvalues["Per Student Budget"].map("${:,.2f}".format)
sortedvalues["Total School Budget"] = sortedvalues["Total School Budget"].map("${:,.2f}".format)

sortedvalues.head()

,School Type,Total School Budget,Student Population,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Reading Score,Average Math Score
school_name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356.00",1858,$582.00,94.133477,97.039828,91.334769,83.975780,83.061895
Thomas High School,Charter,"$1,043,130.00",1635,$638.00,93.272171,97.308869,90.948012,83.848930,83.418349
Griffin High School,Charter,"$917,500.00",1468,$625.00,93.392371,97.138965,90.599455,83.816757,83.351499
Wilson High School,Charter,"$1,319,574.00",2283,$578.00,93.867718,96.539641,90.582567,83.989488,83.274201
Pena High School,Charter,"$585,858.00",962,$609.00,94.594595,95.945946,90.540541,84.044699,83.839917


## Bottom 5 Performing Schools By Overall Student Passing

In [14]:
sortedvalues.tail()

,School Type,Total School Budget,Student Population,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Reading Score,Average Math Score
school_name,,,,,,,,,
Johnson High School,District,"$3,094,650.00",4761,$650.00,66.057551,81.222432,53.539172,80.966394,77.072464
Hernandez High School,District,"$3,022,020.00",4635,$652.00,66.752967,80.862999,53.527508,80.934412,77.289752
Huang High School,District,"$1,910,635.00",2917,$655.00,65.683922,81.316421,53.513884,81.182722,76.629414
Figueroa High School,District,"$1,884,411.00",2949,$639.00,65.988471,80.739234,53.204476,81.158020,76.711767
Rodriguez High School,District,"$2,547,363.00",3999,$637.00,66.366592,80.220055,52.988247,80.744686,76.842711


In [15]:
# Reading by Grade
# seperated by grade dataframes and collect the data for each grade
all9th = school_data_complete.loc[school_data_complete["grade"]=="9th"]
all10th = school_data_complete.loc[school_data_complete["grade"]=="10th"]
all11th = school_data_complete.loc[school_data_complete["grade"]=="11th"]
all12th = school_data_complete.loc[school_data_complete["grade"]=="12th"]

## Math by Grade

In [16]:
# Math by Grade

math9th = all9th.groupby(["school_name"])["math_score"].mean()
math10th = all10th.groupby(["school_name"])["math_score"].mean()
math11th = all11th.groupby(["school_name"])["math_score"].mean()
math12th = all12th.groupby(["school_name"])["math_score"].mean()

math_by_grade = pd.DataFrame({
    "9th":math9th,
    "10th":math10th,
    "11th":math11th,
    "12th":math12th
})
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading by Grade

In [17]:
# Reading by Grade
read9th = all9th.groupby(["school_name"])["reading_score"].mean()
read10th = all10th.groupby(["school_name"])["reading_score"].mean()
read11th = all11th.groupby(["school_name"])["reading_score"].mean()
read12th = all12th.groupby(["school_name"])["reading_score"].mean()


reading_by_grade = pd.DataFrame({
    "9th":read9th,
    "10th":read10th,
    "11th":read11th,
    "12th":read12th
})
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## School by Spending

In [18]:
# School by Spending
# Create bins
bins = [0, 585, 630, 645, 680]
Spending_Ranges = ["<$585","$585-630","$630-645","$645-680"]

school_summary_df_2["Spending Ranges (Per Student)"]=pd.cut(school_summary_df_2["Per Student Budget"], bins, labels= Spending_Ranges, include_lowest = True)
df = school_summary_df_2.groupby("Spending Ranges (Per Student)")["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()

df["Average Math Score"] = df["Average Math Score"].map("{:,.2f}".format)
df["Average Reading Score"] = df["Average Reading Score"].map("{:,.2f}".format)
df["% Passing Math"] = df["% Passing Math"].map("{:,.2f}".format)
df["% Passing Reading"] = df["% Passing Reading"].map("{:,.2f}".format)
df["% Overall Passing"] = df["% Overall Passing"].map("{:,.2f}".format)


C:\Users\david\AppData\Local\Temp\ipykernel_26320\3954553563.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = school_summary_df_2.groupby("Spending Ranges (Per Student)")["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()


In [19]:
df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## School by Size

In [24]:
# School by Size
# Create bins
binss=[0,1000,2000,5000]
school_size = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]


school_summary_df_2["School Size"]=pd.cut(school_summary_df_2["Student Population"], binss, labels = school_size, include_lowest = True)
df = school_summary_df_2.groupby("School Size")["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()
df

C:\Users\david\AppData\Local\Temp\ipykernel_26320\2359696016.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = school_summary_df_2.groupby("School Size")["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"].mean()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## School by Type

In [21]:
# School by type
school_summary_df_2.groupby(["School Type"])["Average Reading Score","Average Math Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


C:\Users\david\AppData\Local\Temp\ipykernel_26320\3174499232.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  school_summary_df_2.groupby(["School Type"])["Average Reading Score","Average Math Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.896421,83.473852,93.620830,96.586489,90.432244
District,80.966636,76.956733,66.548453,80.799062,53.672208


## Analysis and Trends

- Students from Charter Schools outpreformed students from District Schools in all categories - with 36% more students passing both math and reading.

- Students from the largest categories of school were outpreformed by students from medium and small school scoring over 30% more students passing both math and reading.

- School spending more money per student did not increase the average grade of the students or whether they passed. As spending increased the average grade deacreased. 